## Selenium on footballDB

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [61]:
data = pd.read_csv('aug16_try1.csv')
data['empty'] = np.where(data['len']==0, 1, 0)
print('unfinished: ', data['empty'].sum(), '/ ', len(data))

unfins = data[ data['empty']==1].index

unfinished:  139 /  155


#### Initialize Player List:

In [21]:
player_list = []

#### Create webdriver instance:

In [5]:
driver = webdriver.Firefox()

#### Open up FootballDB webpage to player search page:

In [6]:
first_name = 'Derrick' # Sample first name
last_name = 'Brooks' # Sample last name

# Webdriver opens up player search page on FootballDB
driver.get('https://www.footballdb.com/players/index.html')
        
#time.sleep(2)  
print(first_name, ' ', last_name)

Derrick   Brooks


#### Send player's last name to search text box:

In [7]:
driver.find_element_by_xpath('//*[@id="leftcol"]/div[4]/div[5]/form/input[1]').send_keys(last_name)
last='lastNameSent'

#### Click the submit button to perform search:

In [8]:
driver.find_element_by_xpath('//*[@id="leftcol"]/div[4]/div[5]/form/input[2]').click()
last='submitButton'

#### Turn the pages source code into a BeautifulSoup object to find tables and use pandas to convert into dataframe:

In [9]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
tables = soup.find_all('table')

df = pd.read_html(str(tables[0]))[0]

In [15]:
df.head()

,Player,Pos,College,Teams
0,"Brooks, Aaron",QB,Virginia,"2000-2005 NO, 2006 Oak"
1,"Brooks, Ahmad",DB,Texas,"2002 Buf, NFLE: 2004 Fra"
2,"Brooks, Ahmad",LB,Virginia,"2006-2007 Cin, 2008-2016 SF, 2017 GB"
3,"Brooks, Barrett",OT,Kansas State,"1995-1998 Phi, 1999-2000 Det, 2002 GB, 2004-20..."
4,"Brooks, Bill",WR,Boston University,"1986-1992 Ind, 1993-1995 Buf, 1996 Was"


#### We will add the relevant row to our player list like so:

In [22]:
player = df[ df['Player']==(last_name+', '+first_name)]
player_list.append(player)

#### Search for next person on same page's text box by:
1. Clearing the text box <br>
2. Sending the last name of the current player <br>
3. Clicking the submit button

In [27]:
first_name = 'Reggie'
last_name = 'White'

# Clearing the search field text box
driver.find_element_by_xpath( \
    '/html/body/div[1]/div[1]/div[1]/div[3]/div[5]/form/input[1]') \
    .clear()
    
# Sending the last name of current player
driver.find_element_by_xpath( \
    '/html/body/div[1]/div[1]/div[1]/div[3]/div[5]/form/input[1]') \
    .send_keys(last_name)
    
# Clicking the submit button to search
driver.find_element_by_xpath( \
    '/html/body/div[1]/div[1]/div[1]/div[3]/div[5]/form/input[2]') \
    .click()

In [91]:
playerPos = 'DE'

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
tables = soup.find_all('table')

df = pd.read_html(str(tables[0]))[0]

player = df[ df['Player']==(last_name+', '+first_name)]

if len(player)>1:
    player = player[ player['Pos']==playerPos]
player_list.append(player)

#### Unfortunately there is more than one Reggie White: 

In [36]:
player_list[1]

,Player,Pos,College,Teams
90,"White, Reggie",DE,Tennessee,"1985-1992 Phi, 1993-1998 GB, 2000 Car"
91,"White, Reggie",RB,Oklahoma State,2001 Jax
92,"White, Reggie",DT,North Carolina A&T,"1992-1994 SD, 1995 NE"


In [37]:
player_list[1].drop([91,92], inplace=True)

In [63]:
powerups = pd.read_csv('madden19_powerups.csv')
cols = powerups.columns.drop('Unnamed: 4')
powerups.drop('Name', axis=1, inplace=True)
powerups.columns = cols
powerups['Name'] = powerups['Name'].str[:-8]

df2 = pd.DataFrame(powerups['Name'].str.split())
powerups.loc[:, 'firstName'] = df2.Name.map(lambda x: x[0])
powerups.loc[:, 'lastName'] = df2.Name.map(lambda x: ' '.join(x[1:]))

powerups['POS'].value_counts()

position_dict = {'CB':'DB', 'SS':'DB', 'FS':'DB', 'WR':'WR', 'QB':'QB', 'HB':'RB', 'FB':'RB', 
        'TE':'TE', 'LE':'DE', 'RE':'DE', 'DT':'DT', 'LOLB':'LB', 'ROLB':'LB', 'MLB':'LB', 
        'LG':'OG', 'RG':'OG', 'LT':'OT', 'RT':'OT', 'C':'C'}

powerups['POS2'] = powerups['POS'].map(position_dict)

In [125]:
def get_player_teams(first_name, last_name, playerPos, driver):

    try:
        # Clearing the search field text box
        driver.find_element_by_xpath( \
            '/html/body/div[1]/div[1]/div[1]/div[3]/div[5]/form/input[1]').clear()

        # Sending the last name of current player
        driver.find_element_by_xpath( \
            '/html/body/div[1]/div[1]/div[1]/div[3]/div[5]/form/input[1]').send_keys(last_name)

        # Clicking the submit button to search
        driver.find_element_by_xpath( \
        '/html/body/div[1]/div[1]/div[1]/div[3]/div[5]/form/input[2]').click()
        time.sleep(2)
    
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        tables = soup.find_all('table')

        df = pd.read_html(str(tables[0]))[0]

        player = df[ df['Player']==(last_name+', '+first_name)]

        if len(player)>1:
            player = player[ player['Pos']==playerPos]
        
        return player
    except:
        print("# ERROR:")
        return []

In [136]:
#driver = webdriver.Firefox()

driver.get('https://www.footballdb.com/players/players.html?q=develin')

In [137]:
player_list = []

for player in unfins:
    current = all_data.iloc[player]
    first, last, pos = current['firstName'], current['lastName'], current['POS2']
    
    try:
        player_list.append(get_player_teams(first, last, pos, driver))
    except:
        print('# ERROR #')
    time.sleep(1)

In [143]:
player_list = pd.concat(player_list)
#player_list.to_csv('player_list.csv')

In [184]:
all_data['Player'] = all_data['lastName'] + ', '+ all_data['firstName']

merged_df = pd.merge(all_data, player_list, on='Player', how='outer')

In [190]:
fins = all_data[ all_data['empty']==0].index

driver = webdriver.Firefox()

driver.get('https://www.footballdb.com/players/players.html?q=develin')

In [191]:
player_list2 = []

for player in fins:
    current = all_data.iloc[player]
    first, last, pos = current['firstName'], current['lastName'], current['POS2']
    
    try:
        player_list2.append(get_player_teams(first, last, pos, driver))
    except:
        print('# ERROR #')
    time.sleep(1)

In [ ]:
players2 = pd.concat(player_list2)
player_df = pd.concat([players2.reset_index(drop=True), player_list.reset_index(drop=True)])

In [205]:
player_df.shape

(141, 4)

In [ ]:
powerups['Player'] = powerups['lastName'] + ', '+ powerups['firstName']

In [212]:
all_players = set(powerups['Player'].unique())
some_players = set(player_df['Player'].unique())

In [213]:
C = all_players.difference(some_players)

In [235]:
merged = pd.merge(powerups, player_df, on='Player', how='outer')
merged['Teams'] = merged['Teams'].fillna(' ')
merged['Teams'] = merged['Teams'].str.replace('[^a-zA-Z,]', '')

In [285]:
df = merged.Teams.str.split('\s*,\s*', expand=True)\
                .apply(pd.Series.value_counts, 1).iloc[:, 1:].fillna(0)
    
df['LAR'] = df['LA'] + df['LAR'] 
df['RAMS'] = np.where(df['LAR']+df['Stl']>=1, 1, 0)

df['CHARGERS'] = np.where(df['LAC']+df['SD']>=1, 1, 0)

df.drop(['LA', 'NFLEHam', 'Stl', 'LAC', 'LAR', 'SD'], axis=1, inplace=True)

In [286]:
all_df = pd.concat([merged, df], axis=1)
#all_df.to_csv('all_powerups.csv')

In [284]:
remaining = ['Beckham Jr, Odell', 'Brown, Zach', 'Gurley II, Todd', 'Harris Jr, Chris',
 'Harrison Sr, Damon', 'Johnson, Lane', 'Johnson, Trumaine', 'Jones, Reshad', 'Slay Jr, Darius',
 'Smith, Jimmy', 'Smith, Telvin', 'Smith, Tyron', 'Tate III, Golden', 'Thomas III, Earl',
 'Williams, Trent']

rem_teams = [['NYG'], ['Ten', 'Buf', 'Was'], ['RAMS'], ['Den'], ['NYJ', 'NYG'], ['Phi'],
            ['RAMS', 'NYJ'], ['Mia'], ['Det'], ['Bal'], ['Jax'], ['Dal'], ['Sea', 'Det'], 
            ['Sea'], ['Was']]

In [288]:
all_df.to_csv('aug17_final_d')

,Name,OVR,POS,PS4,firstName,lastName,POS2,Player,Pos,College,...,Oak,Phi,Pit,SF,Sea,TB,Ten,Was,RAMS,CHARGERS
0,Champ Bailey,73,CB,"2,000",Champ,Bailey,DB,"Bailey, Champ",DB,Georgia,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
1,Randy Moss,73,WR,"3,400",Randy,Moss,WR,"Moss, Randy",WR,Marshall,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0,0
2,Michael Vick,73,QB,"4,100",Michael,Vick,QB,"Vick, Michael",QB,Virginia Tech,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0
3,Reggie White,73,LE,"1,900",Reggie,White,DE,"White, Reggie",DE,Tennessee,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,Derrick Brooks,72,ROLB,"2,800",Derrick,Brooks,LB,"Brooks, Derrick",LB,Florida State,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
